In [17]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.cluster import KMeans

In [18]:
# Reading the dataset
data = pd.read_excel('Online Retail.xlsx')
data.head()

InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom

In [19]:
# Displaying information about the dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [20]:
# Removing rows with missing CustomerID
data = data[data['CustomerID'].notnull()]
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   StockCode    406829 non-null  object        
 2   Description  406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   InvoiceDate  406829 non-null  datetime64[ns]
 5   UnitPrice    406829 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      406829 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.9+ MB


In [21]:
# Extracting day from InvoiceDate
data['InvoiceDay'] = data['InvoiceDate'].apply(lambda x: dt.datetime(x.year, x.month, x.day))
data.head()

InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country InvoiceDay  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom 2010-12-01  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom 2010-12-01  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom 2010-12-01  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom 2010-12-01  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom 2010-12-01

In [22]:
# Finding the next day of the last invoice date
pin_date = max(data['InvoiceDay']) + dt.timedelta(1)
pin_date

Timestamp('2011-12-10 00:00:00')

In [23]:
# Calculating TotalSum
data['TotalSum'] = data['Quantity'] * data['UnitPrice']
data.head()

InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country InvoiceDay  \
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom 2010-12-01   
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom 2010-12-01   
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom 2010-12-01   
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom 2010-12-01   
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom 2010-12-01   

   TotalSum  
0     15.30  
1     20.34  
2     22.00  
3     20.34  
4     20.34

In [24]:
# Grouping data by CustomerID and calculating Recency, Frequency, Monetary
rfm = data.groupby('CustomerID').agg({
    'InvoiceDay': lambda x: (pin_date - x.max()).days,
    'InvoiceNo': 'count',
    'TotalSum': 'sum'
})
rfm

InvoiceDay  InvoiceNo  TotalSum
CustomerID                                 
12346.0            326          2      0.00
12347.0              3        182   4310.00
12348.0             76         31   1797.24
12349.0             19         73   1757.55
12350.0            311         17    334.40
...                ...        ...       ...
18280.0            278         10    180.60
18281.0            181          7     80.82
18282.0              8         13    176.60
18283.0              4        756   2094.88
18287.0             43         70   1837.28

[4372 rows x 3 columns]

In [25]:
# Renaming columns
rfm.rename(columns= {
    'InvoiceDay': 'Recency',
    'InvoiceNo': 'Frequency',
    'TotalSum': 'Monetary'
}, inplace=True)
rfm

Recency  Frequency  Monetary
CustomerID                              
12346.0         326          2      0.00
12347.0           3        182   4310.00
12348.0          76         31   1797.24
12349.0          19         73   1757.55
12350.0         311         17    334.40
...             ...        ...       ...
18280.0         278         10    180.60
18281.0         181          7     80.82
18282.0           8         13    176.60
18283.0           4        756   2094.88
18287.0          43         70   1837.28

[4372 rows x 3 columns]

In [26]:
# Binning values for Recency, Frequency, Monetary
r_labels = range(4, 0, -1)
r_groups = pd.qcut(rfm['Recency'], q=4, labels=r_labels)
f_labels = range(1, 5)
f_groups = pd.qcut(rfm['Frequency'], q=4, labels=f_labels)
m_labels = range(1, 5)
m_groups = pd.qcut(rfm['Monetary'], q=4, labels=m_labels)

In [27]:
# Adding R, F, M columns to rfm
rfm['R'] = r_groups.values
rfm['F'] = f_groups.values
rfm['M'] = m_groups.values
rfm

Recency  Frequency  Monetary  R  F  M
CustomerID                                       
12346.0         326          2      0.00  1  1  1
12347.0           3        182   4310.00  4  4  4
12348.0          76         31   1797.24  2  2  4
12349.0          19         73   1757.55  3  3  4
12350.0         311         17    334.40  1  1  2
...             ...        ...       ... .. .. ..
18280.0         278         10    180.60  1  1  1
18281.0         181          7     80.82  1  1  1
18282.0           8         13    176.60  4  1  1
18283.0           4        756   2094.88  4  4  4
18287.0          43         70   1837.28  3  3  4

[4372 rows x 6 columns]

In [28]:
# Creating and fitting KMeans model
X = rfm[['R', 'F', 'M']]
kmeans = KMeans(n_clusters=5, init='k-means++', max_iter=300)
kmeans.fit(X)

c:\Users\REZA\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=5)

In [29]:
# Obtaining cluster labels
kmeans.labels_

array([1, 4, 0, ..., 2, 4, 3])

In [30]:
# Adding KMeans cluster labels to rfm
rfm['kmeans_cluster'] = kmeans.labels_
rfm

Recency  Frequency  Monetary  R  F  M  kmeans_cluster
CustomerID                                                       
12346.0         326          2      0.00  1  1  1               1
12347.0           3        182   4310.00  4  4  4               4
12348.0          76         31   1797.24  2  2  4               0
12349.0          19         73   1757.55  3  3  4               3
12350.0         311         17    334.40  1  1  2               1
...             ...        ...       ... .. .. ..             ...
18280.0         278         10    180.60  1  1  1               1
18281.0         181          7     80.82  1  1  1               1
18282.0           8         13    176.60  4  1  1               2
18283.0           4        756   2094.88  4  4  4               4
18287.0          43         70   1837.28  3  3  4               3

[4372 rows x 7 columns]

In [31]:
# Convert all columns to numeric, errors='coerce' will set non-numeric values to NaN
rfm = rfm.apply(pd.to_numeric, errors='coerce')

# Now you can fill missing values with the mean
rfm = rfm.fillna(rfm.mean())

In [32]:
# Mean values of clusters
rfm[rfm['kmeans_cluster'] == 4].mean()

Recency             13.324970
Frequency          294.449223
Monetary          6672.725914
R                    3.735962
F                    3.908005
M                    3.903226
kmeans_cluster       4.000000
dtype: float64

In [33]:
# Overall mean values
rfm.mean()

Recency             92.581199
Frequency           93.053294
Monetary          1898.459701
R                    2.510979
F                    2.487420
M                    2.500000
kmeans_cluster       1.896157
dtype: float64